# Python for (open) Neuroscience

_Lecture 3.4_ - Data visualization

Luigi Petrucco

Jean-Charles Mariani

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vigji/python-cimec/blob/main/lectures/Lecture3.4_Data-visualization.ipynb)

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

cmaps = {}

gradient = np.linspace(0, 1, 256)
gradient = np.vstack((gradient, gradient))
import matplotlib as mpl


def plot_color_gradients(category, cmap_list):
    # Create figure and adjust figure height to number of colormaps
    nrows = len(cmap_list)
    figh = 0.35 + 0.15 + (nrows + (nrows - 1) * 0.1) * 0.32
    fig, axs = plt.subplots(nrows=nrows + 1, figsize=(6.4, figh))
    fig.subplots_adjust(top=1 - 0.35 / figh, bottom=0.15 / figh,
                        left=0.2, right=0.99)
    axs[0].set_title(f'{category} colormaps', fontsize=14)

    for ax, name in zip(axs, cmap_list):
        ax.imshow(gradient, aspect='auto', cmap=mpl.colormaps[name])
        ax.text(-0.01, 0.5, name, va='center', ha='right', fontsize=10,
                transform=ax.transAxes)

    # Turn off *all* ticks & spines, not just the ones with colormaps.
    for ax in axs:
        ax.set_axis_off()

    # Save colormap list for later.
    cmaps[category] = cmap_list

## Outline


 - Some theory
 - The basic plot _galateo_
 - Interactive plots
 - Configuring `matplotlib`
 - `matplotlib` hacks
 - `seaborn` hacks
 - 3D visualizations with `napari`

A great reference

![tufte_book](https://kurtgippert.cdn.bibliopolis.com/pictures/015820.jpg?v=1498601505)

## The data-to-ink ratio principle

    “Graphical excellence is that which gives 
    the greatest number of ideas in the shortest 
    time with the least ink in the smallest space”

When plotting, always strive to maximise the information-per-ink amount!

### A low hanging fruit: dynamite plots

![dynamite](./files/dynamite.png)

For example, dynamite plots are the worst kind of plot:
Each bar is conveying only 2 numbers with a lot of space and ink! 

In [ ]:
np.random.seed(40)
data1 = np.abs(np.random.normal(0, 2, 30)) + 5
data2 = -data1
data2 -= (np.mean(data2) - np.mean(data1)) 

In [ ]:
plt.figure(figsize=(2, 2))
sns.barplot([data1, data2], errorbar="se")
plt.show()

In [ ]:
plt.figure(figsize=(2, 2))
sns.swarmplot([data1, data2])
plt.show()

    Respect each data point! 
  
They're costly and painful to get. If they're not out in your chart (and/or publicly available) they are lost forever!

## Remove the clutter

Try to keep your plot as clean as possible. Remove boxes. Whitespace is enough to separate panels! Remove axes not conveying information.

In [ ]:
f, ax = plt.subplots(figsize=(1.5, 3))
sns.swarmplot(data1)
# sns.despine(bottom=True)
ax.set(xlabel="Some data", ylabel="Count")
plt.show()

In [ ]:
f, ax = plt.subplots(figsize=(1.5, 3))
sns.swarmplot(data1)
sns.despine(bottom=True)
ax.set(xlabel="Some data", ylabel="Count", xticks=[])
plt.show()

### Avoid redundancy!

Do not use different layers of information in a reduntant way:

In [ ]:
x = np.random.normal(0, 1, 300)
y = x + np.random.normal(0, 1, 300)
plt.figure(figsize=(3,3))
plt.scatter(x, y, c=y)
plt.show()

In [ ]:
x = np.random.normal(0, 1, 300)
y = x + np.random.normal(0, 1, 300)
plt.figure(figsize=(3,3))
plt.scatter(x, y)
plt.show()

### Stratify information in the plot

Generally plots are good if they have a first clear message, and they can be further read to reveal more details:

In [ ]:
plt.figure(figsize=(5, 1.8))
sns.swarmplot([data1, data2], orient="h",zorder=-100,alpha=0.5)
sns.boxplot(pd.DataFrame(dict(data1=data1, data2=data2)), 
            boxprops={'facecolor':'none'}, orient="h")

### Link axes and colors

To convey more in a clearer way, use shared axes across plots!

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(4, 1.8), sharex=True)
sns.swarmplot(data1, orient="h", ax=axs[1])
axs[0].hist(data1)
plt.show()

Also, stick to consistent coloring of data points across panels!

## Colormaps!

Colormaps do not only have to be beautiful, but also informative

Conveying info through color can be tricky

### Perceptual linearity

Perceptual linearity is important! That's why you should avoid jet colormaps!

In [ ]:
import numpy as np
from scipy.ndimage import gaussian_filter
np.random.seed(42)
img = np.random.normal(0, 1, (300, 300))
img = gaussian_filter(-img, 50)

In [ ]:
f, axs = plt.subplots(2,2,figsize=(6, 6))
for i, lims in enumerate([(0.008, None), (0.01, -0.01)]):
    for j, cmap in enumerate(["jet", "pink"]):
        axs[i, j].imshow(img, cmap=cmap, vmax=lims[0], vmin=lims[1])
        axs[i, j].axis("off")  # remove axes all around

### Advanced colormap libraries

There's libraries offering more colormaps than default ones, with special attention to perceptual linearity:

- [seaborn](https://seaborn.pydata.org/tutorial/color_palettes.html)
- [colorcet](https://colorcet.holoviz.org/user_guide/index.html)
- [oceancm](https://matplotlib.org/cmocean/)

### Colormap choice

**sequential**: Stick to sequential colormaps for monotonic, positive (or negative)- only data

In [ ]:
plot_color_gradients('Sequential',
                     ['Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds',
                      ])

**Diverging**: Diverging colormaps represent quantities departing from an origin. Make sure you make the white 
the zero of the image!

In [ ]:
plot_color_gradients('Diverging',
                     ['PiYG', 'PRGn', 'BrBG', 'PuOr', 'RdGy', 'RdBu', 'RdYlBu',
                      'RdYlGn', 'Spectral', 'coolwarm', 'bwr'])

**Cyclic**: Colormaps where both edges have same values. Useful for circular data (orientation, angles, phase...)

In [ ]:
plot_color_gradients('Cyclic', ['twilight', 'twilight_shifted', 'hsv'])

Or from seaborn:

In [ ]:
sns.husl_palette(as_cmap=True)

# `matplotlib` pro usage

Let's introduce a bit more `matplotlib` and some more advanced tools there!

### Backends

Default Python would be text-only

To visualize images, we have to rely on <span style="color:indianred">backends</span>!

We can generate plots in a number of ways!

 - as panels opening up between notebooks' cells
 - as windows popping put when running a script
 - directly writing them as files
 - ...

Each of those use entirely different methods to render pixels on the screen...The backend is what takes care of converting plotting instructions into something that we can visualize

### Warning

Backends are often interfaces between Python code and other languages/programs/frameworks! Configuring/switching them sometimes can be confusing. 

Be patient and google them out!

### Interactive plots with different backends

Inside a notebook, we can specify different backends by using the magic notebook command
```python
%matplotlib name_of_the_backend
```

I recommend to use it only once at the beginning of the notebook!

In [ ]:
## This is the default backend, which we get if we do not specify anything:
%matplotlib inline

In [ ]:
plt.figure()
plt.plot(np.random.normal(0, 1, 500))
plt.show()

### Other backends for interactive plots

In [ ]:
## This is the default interactive tool in jupyter notebooks
%matplotlib notebook

In [ ]:
# This is the best interactive backend. It requires to pip install ipympl.
# (generally works better on jupyter lab - a fancier version of jupyter notebook)
%matplotlib widget

## Practical plot hacks

### Use matplotlib configuration tools!

We can change settings for all the plots that we are about to make in a notebook/script using configuration functions!

Check out [here](https://matplotlib.org/stable/tutorials/introductory/customizing.html) a complete list of parameters you can change (the `matplotlibrc` file)

In [ ]:
# For example, this switches permanently off the bounding box:
plt.rc('axes.spines', bottom=True, left=True, right=False, top=False)
plt.figure(figsize=(4, 2))
plt.plot(data1)
plt.show()

In [ ]:
# We can change fonts, fontsizes, etc.
plt.rc('font', size=14, family="serif", serif="Palatino")
plt.figure(figsize=(4, 2))
plt.plot(data1)
plt.show()

In [ ]:
# This restores defaults
plt.rcdefaults()

### Configuration files

Matplotlib has a bunch of different styles that you can choose from. Check them out [here](https://matplotlib.org/stable/gallery/style_sheets/style_sheets_reference.html)!

In [ ]:
# To switch styles:
plt.style.use('dark_background')

In [ ]:
plt.figure(figsize=(4, 3))
plt.plot(data1)
plt.scatter(np.arange(len(data1)), data1, c="C1")
plt.show()

plt.style.use('default') # restore default

## Custom configuration files

We can also define custom style sheets! Highly advisable if we are generating figures for a paper across different scripts/notebooks. 

E.g., the `./lectures/files/example-stylesheet.mplstyle` contains the following:

    font.size : 14
    font.family : serif 
    font.serif : Palatino

In [ ]:
plt.style.use('./files/example-stylesheet.mplstyle')  # use custom stylesheet
plt.figure(figsize=(4, 2))
plt.plot(data1)
plt.show()

plt.rcdefaults()  # restore default

### Color names

Refer to colors in `matplotlib`:

- default colors: `"C0"`, `"C1"`, `"C2"`...

- shades of gray: `".3"` `".5"` `".7"`...

- color name: `"red"`, `"green"`, `"maroon"`, `"teal"`...(list [here](https://matplotlib.org/stable/gallery/color/named_colors.html))
- normalized rgb value (between 0 and 1): `(0.8, 0.1, 0.2)`...

In [ ]:
plt.figure(figsize=(4, 1))
plt.plot(data1, c="teal")
plt.plot(data2, c=(0.8, 0.1, 0.2))
plt.show()

### Zorder

Use the `zorder=` argument to correct appearance overlapping

In [ ]:
plt.figure(figsize=(4, 3))
plt.plot(data1)
plt.scatter(np.arange(len(data1)), data1, 
           c="C1")
plt.show()

In [ ]:
plt.figure(figsize=(4, 3))
plt.plot(data1)
plt.scatter(np.arange(len(data1)), data1, 
           c="C1", zorder=2)
plt.show()

### `plt.axvline()`/`plt.axhline()`

For addin thresholds, onsets, etc.

In [ ]:
plt.figure(figsize=(5, 3))
plt.plot(np.random.normal(0, 1, 500))
plt.axvline(300, c=".6")  # vertical line
plt.axhline(1, c="k", linestyle="dashed")  # horizontal line
plt.show()

### `plt.axhspan()`/`plt.axvspan()`

Highlight areas in the plot

In [ ]:
plt.figure(figsize=(5, 3))
plt.plot(np.random.normal(0, 1, 500))
plt.axhspan(-1, -2, alpha=0.4)
plt.axvspan(100, 200, fc=".6", ec="C1")  # fc: facecolor, filling; ec: edgecolor, line
plt.show()

### Add labels on a plot

We can annotate our plot with text and text boxes!

In [ ]:
fig = plt.figure(figsize=(3, 1.5))
plt.text(0.4, 0.8, "Python's awesome!", size=10)

plt.text(0.4, 0.4, "Python's awesome!", size=10, rotation=20.0,
         ha="center", va="center",  # horizontal and vertical alignment e/r/t keypoint
         bbox=dict(boxstyle="round",  # some infos on the text box
                  ec="none",fc=(1.0, 0.8, 0.8)
    ))

plt.show()

### Custom colormaps

We have seen colormaps when we learned about imshow. Matplotlib has many of them (see [here]( https://matplotlib.org/stable/tutorials/colors/colormaps.html )\)!

If you need a custom colormap, eg to integrate the plot better with an existing color scheme, you can create custom color maps! 

In [ ]:
# Create new colormap with the LinearSegmentedColormap.from_list() method:
import matplotlib.colors
cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["C0", (1,1,1), "orange"])

plt.figure(figsize=(3, 3))
plt.imshow(np.random.normal(0, 1, (50, 50)), cmap=cmap)
plt.colorbar(aspect=5, fraction=0.05, label="Values")  # add colorbar with custom aspect ratio and size
plt.show()

Usually it is better to rely on cmaps designed by experts to be isoluminant!

## `plt.subplots()`

For granular control over a plot's appearance, it can be usefult to open new figures with the `plt.subplots()` function with no arguments. This will create only one axis, but give us the object as well.

We can specify many things in the `gridspec_kw` dictionary argument!

In [ ]:
f, ax = plt.subplots(figsize=(3, 3))
ax.set(xlabel="Something", ylabel="Something else")

## Control multiple plots ratios

In [ ]:
f, ax = plt.subplots(2, 3, figsize=(3, 3), 
         gridspec_kw=dict(left=0.1,  # left padding; also right, top, bottom abailable
                          width_ratios=[0.5, 1, 1],  # ratios of the plots, also "height_ratios"
                          hspace=0,  # space between plots over height; also wspace for width space
                          ))
# ax.set(xlabel="Something", ylabel="Something else")

## From plots to figures

General recommandation:

    Do **not** edit your figures in 
    Inkscape/illustrator after generating them!

Data changes all the times (new inclusions/exclusions, different preprocessing...). You do not want to manually edit figures multiple times!

### A good general approach

Fix as much as possible already on the plot. This way if you have to make a similar figure or update if with new data (or new pipelines) it's 0 time.

Illustrator can be use to compose the final figure just linking individual panels! In this way, consistent fontsizes, colors, linewidths, etc are ensured.

## Export figures

In [ ]:
f, axs = plt.subplots()
axs.set(xlabel="Something", ylabel="Else")

f.savefig("/Users/vigji/Desktop/testfig.pdf")  # save

## Ensure Illustrator/inkscape compatible text:

If we directly open those pdfs in illustrator, we will have funny text (each letter a different box). To fix we have to edit matplotlib config (you can do it in your custom style-sheet!)

In [ ]:
plt.rc("pdf", fonttype=42)  # I swear the number to put here is actually 42...
plt.rc("ps", fonttype=42)

In [ ]:
f.savefig("/Users/vigji/Desktop/testfig.pdf")

## `seaborn`

 - Great library for quick visualizations

 - super well integrated with `pandas`. works very well for datasets having the right shape. (Can be clunky otherwise)

 - we will look at only some functions. You should check out more in the [gallery](https://seaborn.pydata.org/examples/index.html)

### Basic seaborn plotting functions

### `sns.swarmplot()`

Nice visualization of many datapoints with self-adjusted spreading

In [ ]:
plt.figure(figsize=(3, 1))
sns.swarmplot(pd.DataFrame(dict(data1=data1, data2=data2)), orient="h")

### `sns.boxplot()`

Customizable boxplot. Adjusting properties can be a bit tricky, see example above in the intro!

In [ ]:
plt.figure(figsize=(3, 1))
sns.boxplot(pd.DataFrame(dict(data1=data1, data2=data2)), orient="h")

### `sns.violinplot()`

Kernel density estimation (KDE) is a way of estimating a continuous probability distribution from many many samples (my recommandation: use only with > 100s of points!)

In [ ]:
plt.figure(figsize=(3, 1))
sns.violinplot(pd.DataFrame(dict(data1=data1, data2=data2)), orient="h")

### `sns.barplot()`

Use this ONLY if you are showing also individual data points!

In [ ]:
plt.figure(figsize=(4, 1.5))
ax = sns.barplot(pd.DataFrame(dict(data1=data1, data2=data2)), 
                 capsize=0.1,  # wisker width
                 errwidth=1,  # error lines width
                 orient="h")  # horizontal orientation
for bars in ax.containers:  # Loop over bars, and set color of the edges instead of color of the filling
    for i, bar in enumerate(bars):
        bar.set_edgecolor(f"C{i}")
        bar.set_facecolor("None")
sns.swarmplot(pd.DataFrame(dict(data1=data1, data2=data2)), orient="h", alpha=0.4, zorder=-100)

### Advanced, multi-layer plots

`seaborn` offers also some complex, multi-element (and multi-axis) tool for composite visualizations. Explore them in the gallery, but beware! They tend to work only if you need exctly what they offer!

### lineplot with error bars

In [ ]:
fmri = sns.load_dataset("fmri")
fmri.head()

In [ ]:
sns.lineplot(data=fmri, x="timepoint", y="signal", hue="event")

### lineplot with colors based on feature

In [ ]:
dots = sns.load_dataset("dots").query("align == 'dots'")
dots.head()

In [ ]:
sns.lineplot(
    data=dots, x="time", y="firing_rate", hue="coherence", style="choice",
)

### Multiple linear regression plots

In [ ]:
penguins = sns.load_dataset("penguins")
penguins.head()

In [ ]:
g = sns.lmplot(
    data=penguins,
    x="bill_length_mm", y="bill_depth_mm", hue="species",
    height=3
)

### KDE joint plot

In [ ]:

f = plt.figure()
# Show the joint distribution using kernel density estimation
g = sns.jointplot(
    data=penguins,
    x="bill_length_mm", y="bill_depth_mm", hue="species",
    kind="kde",
    height=4
)

## `napari`

Napari is a truly awesome tool for 3D data/video visualization and annotation in Python

Developed by great people with state-of-the-art Python and open-source community management!

Works very well also integrated with notebooks.

Use `napari` from notebooks

In [ ]:
# Before starting, conda install "napari[all]". Pip installation could have issues on macOS!

# we allow the notebook to handle a parallel event loop for the interface (see last lecture!)
%gui qt5

In [ ]:
import napari
import nibabel

In [ ]:
img = nibabel.load("/Users/vigji/Downloads/imaging_data/sub-001/ses-1/anat/sub-001_ses-1_acq-RARE_T2w.nii.gz")
stack = img.get_fdata()

In [ ]:
viewer = napari.Viewer()
viewer.add_image(stack)

In [ ]:
points = viewer.layers["points_layer"]

In [ ]:
points.data

## Course wrapup

## Take home messages

 - Use Python where you can...

- But if you don't, don't use MATLAB for statistics!

- Use GitHub, and publish your code (not on GitHub)

- Blow up dynamite plot! Together we can!

### Deadlines for the assignments



 - **15 Jul 2023** for the Assignment 0 and Assignment 1
 - **08 Sep 2023** for the project (pending Leah's confirmation)

## And now... 

##                🍻 ...beer! 🍻